# 深層学習ノートブック-6 誤差逆伝播(Backpropagation)
誤差逆伝播をスクラッチで実装してみる   
* $l$番目の層における損失の誤差 
$$\delta^{[l]}=\frac{\partial{L}}{\partial{Z^{[l]}}}$$  
$$=\frac{\partial{L}}{\partial{A^{[l]}}} \bigodot \frac{\partial{A}}{\partial{Z^{[l]}}}$$
$$=(\delta^{[l+1]}\bm{W}^{[l+1]}) \bigodot \sigma'(\bm{Z^{[l]}})$$  

* パラメタの勾配
$$\frac{\partial{L}}{\partial{W^{[l]}}}=\delta^{[l]T}\bm{A}^{[l-1]}$$  
$$\frac{\partial{L}}{\partial{b^{[l]}}}=\sum_i\delta_i^{[l]}$$  

* パラメタの更新
$$\bm{W^{[l]}}=\bm{W^{[l]}}-\alpha\frac{\partial{L}}{\partial{W^{[l]}}}$$  
$$\bm{B^{[l]}}=\bm{B^{[l]}}-\alpha\frac{\partial{L}}{\partial{B^{[l]}}}$$  


※ReLUの導関数
\begin{equation}
\sigma^{'}(z)= \left \{
\begin{array}{l}
1　(if z > 0) \\
0　(otherwise)
\end{array}
\right.
\end{equation}

In [23]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

In [22]:
# 線形変換部分(パラメタ)の勾配の算出関数。Aはl-1層目の値で他はl層目の値。
def linear_backward(A, W, b, Z):
     # pytorch.tensorの.grad属性と区別するために.grad_として属性を定義している
     W.grad_ = Z.grad_ @ A
     # bの値が一つ変わると、全データ分Lへ影響を及ぼすので、行方向(データ数の方向)にZ.grad_を足す。
     b.grad_ = torch.sum(Z.grad_, dim=0)
     # l-1層目のAはl層目の誤差（=Z.grad_）とl層目のWから計算できる。
     A.grad_ = Z.grad_ @ W


# ReLU関数
def relu_backward(Z, A):
     # 上式の"l番目の層における損失の誤差"における2行目と3行目を組み合わせてdL/dZを算出している
     # A.grad_はdL/dA, (Z > 0).float()はReLu関数の偏微分に相当する。
     Z.grad_ = A.grad_ * ( Z > 0 ).float()

### 補足1
tensor * (tensor > 0)の計算について

In [20]:
a = torch.randn((2,3)) 
b = torch.randn((2,3)) 
print(a)
print(b)
print(b > 0)

tensor([[ 0.8516, -1.0509,  0.1294],
        [-0.1999,  1.1394,  1.4127]])
tensor([[ 0.1299,  0.2585, -1.2203],
        [ 0.6629, -0.2176, -1.2055]])
tensor([[ True,  True, False],
        [ True, False, False]])


In [21]:
a * (b > 0)

tensor([[ 0.8516, -1.0509,  0.0000],
        [-0.1999,  0.0000,  0.0000]])

このようにb > 0がTrueの要素は1、Falseの要素は0として計算される。

上記のように損失、パラメタの勾配をコードで表わすことが出来た。  
ではこれらの計算を実際に実行するにはどうすればよいか？  
当然、具体的な計算を行うためには上記の関数の入力であるA,W,b,Zを求める必要がある。  
すなわち順伝播(forward)の計算を行い、損失を計算してから逆伝播(backward)の計算を行うことになる。  

まず損失を求める関数を実装する。

In [24]:
# softmax関数と交差エントロピーの計算を同じ関数で実装する（pytorchでもこうなっている）
def softmax_cross_entropy(X, y_true):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    '''
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10
    softmax_out = e_x / denominator
    loss = - (y_true * torch.log(softmax_out + 1e-10)).sum() / y_true.shape[0]

    return loss, softmax_out


In [25]:
# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B

In [26]:
# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)

順伝播→逆伝播の計算を行う関数を定義。  

In [28]:
# 隠れ層１層の場合のMLP
def forward_and_backward(X, W_list, B_list, y):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    #########################
    #### 順伝播（forward）####
    #########################

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_list[0], B_list[0])

    # 活性化関数適用
    A1 = ReLU(Z1)

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_list[1], B_list[1])

    # 最終出力。損失とソフトマックス関数の出力が返される
    loss, A2 = softmax_cross_entropy(Z2, y)


    #########################
    #### 逆伝播（backward）####
    #########################

    # 最終出力→出力層の出力のbackward。
    # ここでは出力層の活性化関数は恒等関数とし、
    # その出力にsoftmax_cross_entropyを適用した結果をモデルの最終出力として考えている。
    Z2.grad_ = (A2 - y) / X.shape[0]
    # 出力層→隠れ層のbackward
    linear_backward(A1, W_list[1], B_list[1], Z2)
    # 隠れ層の出力側のbackward
    relu_backward(Z1, A1)
    # 隠れ層の入力側のbackward
    linear_backward(X, W_list[0], B_list[0], Z1)
    
    return loss, Z1, A1, Z2, A2
    

### 補足：Z2.grad_をデータ数X.shape[0]で割る理由  
交差エントロピーで算出した全体の損失Lはデータ数で割った平均の形なので、右辺もデータ数で割る必要あり。  
各データに対応する損失をデータ数で割ることで、最終的な全体の損失Lへ与える影響をならしているイメージ。  
誤差逆伝播の講義資料のdL/dZ[2]の式を参照。